In [ ]:
#######################################################################
##  Made by: Dr. Keungoui Kim
##  Title: Scitech data import from Dimensions
##  goal : 
##  Data set:  
##  Time Span:
##  Variables
##      Input: 
##      Output: 
##  Time-stamp: #  "Sun Jan 26 01:47:34 2020":  edited by awe kim ; code
##  Notice :
#######################################################################

In [120]:
### Library
import pandas as pd
import numpy as np
import math

In [121]:
### Directory Setting for Export
#dir = "E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/" # Home
dir = "D:/Google Drive(awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/" # HGU

In [122]:
### Data Import
import dimcli
dimcli.login(key="792DDFAFCCA7478D8F37159F274A2783",
             endpoint="https://app.dimensions.ai/api/dsl/v2")

dsl = dimcli.Dsl()

Dimcli - Dimensions API Client (v0.9.9.1)
Connected to: <https://app.dimensions.ai/api/dsl/v2> - DSL v2.5
Method: manual login


In [156]:
### Extract patent data
pat_data = dsl.query_iterative("""search patents
                                  where publications is not empty and year = 2022                    
                                  return patents[id+year+priority_year+cpc+publications+publication_ids]""") #  limit 50000

Starting iteration with limit=1000 skip=0 ...
0-1000 / 12332 (10.04s)
1000-2000 / 12332 (5.32s)
2000-3000 / 12332 (4.10s)
3000-4000 / 12332 (4.69s)
4000-5000 / 12332 (4.85s)
5000-6000 / 12332 (6.19s)
6000-7000 / 12332 (4.84s)
7000-8000 / 12332 (4.53s)
8000-9000 / 12332 (4.95s)
9000-10000 / 12332 (5.99s)
10000-11000 / 12332 (8.05s)
11000-12000 / 12332 (3.11s)
12000-12332 / 12332 (2.19s)
===
Records extracted: 12332


In [161]:
pat_data_df = pat_data.as_dataframe()
print(pat_data_df.shape)
print(pat_data_df.id.nunique())

(12332, 6)
12332


In [162]:
pat_data_df.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pat_data_df_"+"2022"+".csv", index=False)
pat_data_df.head()

,id,cpc,priority_year,publication_ids,publications,year
0,WO-2022236335-A1,"[A61K38/177, A61P29/00, C07K14/70578, A61P37/0...",2021.0,"[pub.1027342753, pub.1009940936]","[{'doi': '10.1182/blood-2003-06-2031', 'id': '...",2022
1,WO-2022236333-A2,"[A61B3/0016, A61B3/0033]",2021.0,[pub.1065228315],"[{'doi': '10.1364/ol.35.000739', 'id': 'pub.10...",2022
2,WO-2022236331-A1,"[B05B12/18, B05B12/16, B05B14/00, B05B7/0408]",2021.0,[pub.1103595211],"[{'doi': '10.1002/adem.201701084', 'id': 'pub....",2022
3,WO-2022236308-A1,"[A61B5/0042, A61B34/30, G01R33/3806, G01R33/34...",2021.0,[pub.1092392383],"[{'doi': '10.1109/tmag.2017.2751001', 'id': 'p...",2022
4,WO-2022236297-A1,"[C11D3/48, C11D3/0068, C11D3/381, C11D7/40, C1...",2021.0,[pub.1080312358],"[{'doi': '10.1128/jb.96.2.479-486.1968', 'id':...",2022


In [25]:
### Import .csv files
# pat_data_df = pd.read_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pat_data_df_01.csv")

# from ast import literal_eval

# pat_data_df['publication_ids'] = pat_data_df['publication_ids'].apply(literal_eval) #convert to list type

In [163]:
pat_pub_df = pat_data_df[['id','publication_ids']]
pat_pub_df = pat_pub_df.explode("publication_ids", ignore_index=True)
pat_pub_df.head()

,id,publication_ids
0,WO-2022236335-A1,pub.1027342753
1,WO-2022236335-A1,pub.1009940936
2,WO-2022236333-A2,pub.1065228315
3,WO-2022236331-A1,pub.1103595211
4,WO-2022236308-A1,pub.1092392383


In [164]:
### Extract publication_ids
pub_list = pat_pub_df.publication_ids.unique()
pd.DataFrame(pub_list, columns=["publication_ids"]).to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pub_list_df_"+"2022"+".csv")

## Finalized Version
- Maximum 50,000 records
- dimcli query cannot recognize ''... so I had to write the loop for each jurisdiction --> fixed
- If I could convert dataframe's column into a big list with "" as elements, I would use Version 2.

### Iterative Codes for extracting patent data

In [ ]:
### Create patent data.frame
years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022'] 
offices = ['WO'] # 'WO', 'US', 'EP'

for year in years:
    for office in offices:
        ### Patent set
        query = """search patents 
                    where publications is not empty and jurisdiction = """+ '''"'''+office+'''"''' +""" and priority_year=""" + year + """ 
                    return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids] sort by priority_date"""
        pat_data = dsl.query_iterative(query) 
        pat_df = pat_data.as_dataframe()

        # Export patent set
        pat_df.to_csv(dir+"pat_data_df_"+year+"_"+office+".csv", index=False)

### Iterative Codes for extracting publication data

In [164]:
### Create patent data.frame
years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022'] 
offices = ['EP']

for year in years:
    for office in offices:
        ### Patent set
        query = """search patents 
                    where publications is not empty and jurisdiction = """+ '''"'''+office+'''"''' +""" and priority_year=""" + year + """ 
                    return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids] sort by priority_date"""
        pat_data = dsl.query_iterative(query) 
        pat_df = pat_data.as_dataframe()

        # Export patent set
        pat_df.to_csv(dir+"pat_data_df_"+year+"_"+office+".csv", index=False)

        # Create publication set
        pat_pub_df = pat_df[['id','publication_ids']]
        pat_pub_df = pat_pub_df.explode("publication_ids", ignore_index=True)

        ## Export Publication List version 2. 50,000 each
        pub_list = pat_pub_df.publication_ids.unique()
        pub_list = pd.DataFrame(pub_list, columns=["publication_ids"])
        loop = math.ceil(pub_list.shape[0]/50000)+1
        for i in range(1,loop):
            temp = pub_list.iloc[0:49999,]
            temp.to_csv(dir+"pub_list_df_"+year+"_"+office+str(i)+".csv", index=False)

            # in operator only can contain maximum 512 elements
            # Thus, I run another loop for extracting 500 publication ids
            inloop = math.ceil(temp.shape[0]/500)+1
            pub_df = pd.DataFrame() # Create empty data.frame  
            for j in range(1, inloop):
                intemp = temp.iloc[0:499,]
                query = """search publications 
                        where id in """+'''['''+','.join(f'"{x}"' for x in intemp['publication_ids'].to_list())+''']'''+"""
                        return publications[id+type+volume+year+issue+title+journal+authors+category_for]"""
                intemp_data = dsl.query_iterative(query)
                intemp_data = intemp_data.as_dataframe()    
                
                pub_df = pd.concat([pub_df, intemp_data], axis=0)
                temp = temp.drop(temp.index[0:499])          


            # Save publication set
            pub_df.to_csv(dir+"pub_data_df_"+year+"_"+office+"_"+str(i)+".csv", index=False)
            pub_list = pub_list.drop(pub_list.index[0:49999])          

    print(year, office)        

Starting iteration with limit=1000 skip=0 ...
0-1000 / 18981 (2.20s)
1000-2000 / 18981 (5.89s)
2000-3000 / 18981 (3.07s)
3000-4000 / 18981 (1.92s)
4000-5000 / 18981 (2.02s)
5000-6000 / 18981 (1.82s)
6000-7000 / 18981 (1.95s)
7000-8000 / 18981 (4.71s)
8000-9000 / 18981 (5.98s)
9000-10000 / 18981 (2.35s)
10000-11000 / 18981 (4.10s)
11000-12000 / 18981 (5.51s)
12000-13000 / 18981 (1.91s)
13000-14000 / 18981 (2.13s)
14000-15000 / 18981 (6.02s)
15000-16000 / 18981 (2.05s)
16000-17000 / 18981 (1.99s)
17000-18000 / 18981 (1.93s)
18000-18981 / 18981 (2.00s)
===
Records extracted: 18981
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (13.41s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (7.55s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (5.72s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (3.56s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 

2010 EP


0-1000 / 19220 (3.68s)
1000-2000 / 19220 (5.99s)
2000-3000 / 19220 (4.21s)
3000-4000 / 19220 (3.81s)
4000-5000 / 19220 (5.36s)
5000-6000 / 19220 (5.29s)
6000-7000 / 19220 (8.04s)
7000-8000 / 19220 (2.59s)
8000-9000 / 19220 (6.61s)
9000-10000 / 19220 (7.52s)
10000-11000 / 19220 (4.92s)
11000-12000 / 19220 (2.37s)
12000-13000 / 19220 (2.60s)
13000-14000 / 19220 (9.03s)
14000-15000 / 19220 (4.32s)
15000-16000 / 19220 (2.44s)
16000-17000 / 19220 (6.25s)
17000-18000 / 19220 (5.93s)
18000-19000 / 19220 (2.38s)
19000-19220 / 19220 (5.45s)
===
Records extracted: 19220
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (10.00s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (4.30s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (6.43s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (4.22s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (4

2011 EP


0-1000 / 17162 (3.45s)
1000-2000 / 17162 (3.51s)
2000-3000 / 17162 (4.15s)
3000-4000 / 17162 (3.96s)
4000-5000 / 17162 (4.63s)
5000-6000 / 17162 (2.68s)
6000-7000 / 17162 (5.42s)
7000-8000 / 17162 (2.74s)
8000-9000 / 17162 (2.35s)
9000-10000 / 17162 (3.57s)
10000-11000 / 17162 (2.67s)
11000-12000 / 17162 (3.31s)
12000-13000 / 17162 (4.54s)
13000-14000 / 17162 (2.66s)
14000-15000 / 17162 (2.66s)
15000-16000 / 17162 (5.59s)
16000-17000 / 17162 (3.05s)
17000-17162 / 17162 (1.73s)
===
Records extracted: 17162
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (5.76s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (6.52s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (4.90s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (5.37s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (6.21s)
===
Records extracted: 499
Starting iteration with 

...

2020 EP


0-1000 / 2486 (2.04s)
1000-2000 / 2486 (5.98s)
2000-2486 / 2486 (2.91s)
===
Records extracted: 2486
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (4.35s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (5.56s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (4.56s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (4.88s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (4.51s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (4.64s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (6.80s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (5.21s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (5.73s)
===
Records extracted: 499
Starting iteration with limit=1000 skip=0 ...
0-499 / 499 (4.61

2021 EP


0-1 / 1 (1.27s)
===
Records extracted: 1
Starting iteration with limit=1000 skip=0 ...
0-3 / 3 (2.51s)
===
Records extracted: 3


2022 EP


### Original version

In [ ]:
### Create publication list
years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022'] 
offices = ['EP']

for year in years:
    for office in offices:
        # Create publication set
        pat_pub_df = pat_df[['id','publication_ids']]
        pat_pub_df = pat_pub_df.explode("publication_ids", ignore_index=True)

        ## Export Publication List version 1. ALL
        pub_list = pat_pub_df.publication_ids.unique()
        pub_list = pd.DataFrame(pub_list, columns=["publication_ids"])
        pub_list.to_csv(dir+"pub_list_df_"+year+"_"+office+".csv", index=False)

        ## Export Publication List version 2. 50,000 each
        pub_list = pat_pub_df.publication_ids.unique()
        pub_list = pd.DataFrame(pub_list, columns=["publication_ids"])
        loop = math.ceil(pub_list.shape[0]/50000)+1
        for i in range(1,loop):
            temp = pub_list.iloc[0:49999,]
            temp.to_csv(dir+"pub_list_df_"+year+"_"+office+str(i)+".csv", index=False)
            pub_list = pub_list.drop(pub_list.index[0:49999])          

In [ ]:
### Create publication data.frame
years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022'] 
offices = ['EP']

for year in years:
    for office in offices:
        ### Publication set

        ## Version 1. Request query for single publication id --> takes longer time
        pub_list = pd.read_csv(dir+"pub_list_df_"+year+"_"+office+".csv")
        # First record
#        query = """search publications 
#                    where id = """+'''"'''+pub_list.publication_ids[0]+'''"'''+"""
#                    return publications[id+type+volume+year+issue+title+journal+authors]"""
#        temp_data = dsl.query(query)    
#        pub_df = temp_data.as_dataframe()

#        for pub in range(1,len(pub_list.publication_ids)):
#            query = """search publications 
#                        where id = """+'''"'''+pub_list.publication_ids[pub]+'''"'''+"""
#                        return publications[id+type+volume+year+issue+title+journal+authors+category_for]"""
#            temp_data = dsl.query(query)    
#            temp_data = temp_data.as_dataframe()
#            
#           # bind rows
#           pub_df = pd.concat([pub_df, temp_data], axis=0)
#
#           print(year, office, pub)

        # Version 2.
        query = """search publications 
                   where id in """+'''['''+','.join(f'"{x}"' for x in pub_list['0'].to_list())+''']'''+"""
                   return publications[id+type+volume+year+issue+title+journal+authors+category_for]"""
        temp_data = dsl.query_iterative(query)
        pub_df = temp_data.as_dataframe()    

        # Save publication set
        pub_df.to_csv(dir+"pub_data_df_"+year+"_"+office+".csv")

### Load Publication list manually


In [ ]:
import os
os.listdir(dir+"/")

### Check Total Counts

In [ ]:
### Check total counts

years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022'] 
totalRecUS = []
totalRecEU = []
totalRecWO = []
for year in years:
    ### US
    query = """search patents 
                    where publications is not empty and jurisdiction in ["US"] and priority_year=""" + year + """ 
                    return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids] sort by priority_date"""
    pat_data = dsl.query(query) 
    totalRecUS.append(pat_data.count_total)

    ### EP
    query = """search patents 
            where publications is not empty and jurisdiction in ["EP"] and priority_year=""" + year + """ 
            return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids] sort by priority_date"""
    pat_data = dsl.query(query) 
    totalRecEUWO.append(pat_data.count_total)

    ### WO
    query = """search patents 
            where publications is not empty and jurisdiction in ["WO"] and priority_year=""" + year + """ 
            return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids] sort by priority_date"""
    pat_data = dsl.query(query) 
    totalRecEUWO.append(pat_data.count_total)

    print(year)

# Present result
pd.DataFrame({'year':years,'USRec':totalRecUS,'EUWORec':totalRecEUWO})

#### Alternative codes

In [ ]:
years = '2021'

for year in years:
    query = """search patents 
            where publications is not empty and year=""" + year + """
            return patents[id+year+priority_year+cpc+publications+publication_ids]"""
    pat_data = dsl.query_iterative(query) 
    pat_df = pat_data.as_dataframe()
    rangeIndex = int(temp.count_total/1000)+1

    for t in range(50,rangeIndex):
        query = """search patents 
            where publications is not empty and year=""" + year + """
            return patents[id+year+priority_year+cpc+publications+publication_ids+reference_ids+times_cited] limit 1000 skip """ + str(1000*t)
        temp_data = dsl.query(query)
        temp_df = temp_data.as_dataframe()
        # bind rows
        pat_df = pd.concat([pat_df, temp_df], axis=0)
        print(t)

    # Save patent set
    pat_data_df.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pat_data_df_"+year+".csv", index=False)

    # Save publication set
    pat_pub_df = pat_data_df[['id','publication_ids']]
    pat_pub_df = pat_pub_df.explode("publication_ids", ignore_index=True)

    # Extract publication_ids
    pub_list = pat_pub_df.publication_ids.unique()
    pd.DataFrame(pub_list, columns=["publication_ids"]).to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pub_list_df_"+year+".csv", index=False)

In [ ]:
# %%dslloop my_data search publications for "malaria" return publications limit 500

UsageError: %%dslloop is a cell magic, but the cell body is empty. Did you mean the line magic %dslloop (single %)?


In [ ]:
##### EP #####
    query = """search patents 
                where publications is not empty and jurisdiction in ["EP"] and priority_year=""" + year + """ 
                return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids] sort by priority_date"""
    pat_data = dsl.query_iterative(query) 
    pat_df = pat_data.as_dataframe()

    # Save patent set
    pat_df.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pat_data_df_"+year+"_EP.csv", index=False)

    # Save publication set
    pat_pub_df = pat_df[['id','publication_ids']]
    pat_pub_df = pat_pub_df.explode("publication_ids", ignore_index=True)

    # Extract publication_ids
    pub_list = pat_pub_df.publication_ids.unique()
    pub_list = pd.DataFrame(pub_list, columns=["publication_ids"])

    loop = math.ceil(pub_list.shape[0]/50000)+1
    
    for i in range(1,loop):
        temp = pub_list.iloc[0:49999,]
        temp.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pub_list_df_"+year+"_EP_"+str(i)+".csv")
        pub_list = pub_list.drop(pub_list.index[0:49999])

    ##### WO #####
    query = """search patents 
                where publications is not empty and jurisdiction in ["WO"] and priority_year=""" + year + """ 
                return patents[id+family_id+priority_year+priority_date+jurisdiction+cpc+publication_ids] sort by priority_date"""
    pat_data = dsl.query_iterative(query) 
    pat_df = pat_data.as_dataframe()

    # Save patent set
    pat_df.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pat_data_df_"+year+"_WO.csv", index=False)

    # Save publication set
    pat_pub_df = pat_df[['id','publication_ids']]
    pat_pub_df = pat_pub_df.explode("publication_ids", ignore_index=True)

    # Extract publication_ids
    pub_list = pat_pub_df.publication_ids.unique()
    pub_list = pd.DataFrame(pub_list, columns=["publication_ids"])

    loop = math.ceil(pub_list.shape[0]/50000)+1
    
    for i in range(1,loop):
        temp = pub_list.iloc[0:49999,]
        temp.to_csv("E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/pub_list_df_"+year+"_WO_"+str(i)+".csv")
        pub_list = pub_list.drop(pub_list.index[0:49999])
